In [4]:
#importing libraries
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score
import nltk

stopwords

In [5]:
#downloading stopwords
nltk.download('stopwords')
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


data pre-processing

In [34]:
#importing data file
df=pd.read_csv('/content/train.csv',error_bad_lines=False, engine="python")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
Skipping line 3074: unexpected end of data


In [35]:
df.shape

(3072, 5)

In [8]:
#pre viewing the data
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [9]:
#finding null value
df.isnull().sum()

id          0
title      51
author    209
text        4
label       0
dtype: int64

In [10]:
#filling null value if present
df=df.fillna(' ')

In [11]:
#merging authore name +title
df['content']=df['author']+' '+df['title']

In [12]:
print(df['content'])

0       Darrell Lucus House Dem Aide: We Didn’t Even S...
1       Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo...
2       Consortiumnews.com Why the Truth Might Get You...
3       Jessica Purkiss 15 Civilians Killed In Single ...
4       Howard Portnoy Iranian woman jailed for fictio...
                              ...                        
1970    Author USA – China: Who Is Responsible for Nor...
1971    David Waldstein and Ben Rothenberg U.S. Open Q...
1972    Michelle Innis Australia Is Not as Down Under ...
1973    Poke Staff 7 Tory horror film posters to send ...
1974                                          Anonymous  
Name: content, Length: 1975, dtype: object


In [13]:
#seprating data and label
X=df.drop(columns='label',axis=1)
Y=df['label']

In [14]:
print(X)
print(Y)

        id                                              title  \
0        0  House Dem Aide: We Didn’t Even See Comey’s Let...   
1        1  FLYNN: Hillary Clinton, Big Woman on Campus - ...   
2        2                  Why the Truth Might Get You Fired   
3        3  15 Civilians Killed In Single US Airstrike Hav...   
4        4  Iranian woman jailed for fictional unpublished...   
...    ...                                                ...   
1970  1970  USA – China: Who Is Responsible for North Kore...   
1971  1971  U.S. Open Quieted Those Calling for a Roof. No...   
1972  1972  Australia Is Not as Down Under as Everyone Thi...   
1973  1973  7 Tory horror film posters to send a chill dow...   
1974  1974                                                      

                                  author  \
0                          Darrell Lucus   
1                        Daniel J. Flynn   
2                     Consortiumnews.com   
3                        Jessica Purkiss   

Stemming

In [15]:
port_stem = PorterStemmer()

In [16]:
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [17]:
df['content'] = df['content'].apply(stemming)

In [18]:
print(df['content'])

0       darrel lucu hous dem aid even see comey letter...
1       daniel j flynn flynn hillari clinton big woman...
2                  consortiumnew com truth might get fire
3       jessica purkiss civilian kill singl us airstri...
4       howard portnoy iranian woman jail fiction unpu...
                              ...                        
1970    author usa china respons north korean nuclear ...
1971    david waldstein ben rothenberg u open quiet ca...
1972    michel inni australia everyon think new york time
1973    poke staff tori horror film poster send chill ...
1974                                               anonym
Name: content, Length: 1975, dtype: object


In [19]:
#separating the data and label
X = df['content'].values
Y = df['label'].values

In [20]:
print(X)

['darrel lucu hous dem aid even see comey letter jason chaffetz tweet'
 'daniel j flynn flynn hillari clinton big woman campu breitbart'
 'consortiumnew com truth might get fire' ...
 'michel inni australia everyon think new york time'
 'poke staff tori horror film poster send chill spine halloween' 'anonym']


In [21]:
print(Y)

[1 0 1 ... 0 1 1]


In [22]:
Y.shape

(1975,)

In [23]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()
vectorizer.fit(X)

X = vectorizer.transform(X)

In [24]:
print(X)

  (0, 5106)	0.2988906032626825
  (0, 4365)	0.26169514290578866
  (0, 2931)	0.3370138049505055
  (0, 2839)	0.2988906032626825
  (0, 2551)	0.24736668016881966
  (0, 2327)	0.23193627189537763
  (0, 1652)	0.2516550530981723
  (0, 1278)	0.27463906992696424
  (0, 1211)	0.3370138049505055
  (0, 980)	0.2563956294363691
  (0, 821)	0.3552572454411006
  (0, 98)	0.28284242060994297
  (1, 5413)	0.308265487806722
  (1, 2273)	0.19695229440722908
  (1, 1846)	0.6883566973493513
  (1, 1203)	0.28978569304450547
  (1, 927)	0.20436653604578373
  (1, 738)	0.3880779576281725
  (1, 637)	0.15871756084162852
  (1, 503)	0.3013470519256807
  (2, 5089)	0.4232302699121657
  (2, 3144)	0.48492083192704566
  (2, 2006)	0.35154712234393204
  (2, 1806)	0.35635819757989223
  (2, 1036)	0.48492083192704566
  :	:
  (1971, 3344)	0.09544548619542835
  (1971, 2922)	0.3552855622098234
  (1971, 1708)	0.25898804885092575
  (1971, 1220)	0.21937415405604435
  (1971, 727)	0.2319547590222525
  (1971, 467)	0.2398693899305878
  (1972, 5

Splitting the dataset

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.3,  random_state=2)

Modeling: Logistic Regression

In [26]:
model = LogisticRegression()

In [27]:
model.fit(X_train, Y_train)

LogisticRegression()

accuracy score

In [28]:
# accuracy score on the training data
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [29]:
print('Accuracy score of the training data : ', training_data_accuracy)

Accuracy score of the training data :  0.9696092619392185


In [30]:
# accuracy score on the test data
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [31]:
print('Accuracy score of the test data : ', test_data_accuracy)

Accuracy score of the test data :  0.9460370994940978


Making a Predictive System

In [32]:
X_new = X_test[100]

prediction = model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print('The news is Real')
else:
  print('The news is Fake')

[1]
The news is Fake


In [33]:
print(Y_test[100])

1
